In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

import librosa
from librosa.display import specshow, waveshow

from IPython.display import Audio

In [ ]:
from models.modules.dsp import HarmonicOscillator, FilteredNoise

In [ ]:
sample_rate = 16000
window_length = 1024
hop_length = 64
n_harmonics = 64
n_bands = 128
n_channels = 2
time_steps = 1000
batch_size = 8

In [ ]:
osc = HarmonicOscillator(sample_rate, hop_length, n_harmonics, n_channels)

In [ ]:
base_pitch_1 = torch.linspace(55, 440, time_steps)
base_pitch_2 = torch.linspace(55.1, 441, time_steps)
base_pitch = torch.concat([base_pitch_1[None, :, None], base_pitch_2[None, :, None]], -1)
base_pitch = torch.tile(base_pitch, (batch_size, 1, 1))
amplitude = torch.ones(batch_size, time_steps, n_channels)

harmonic_distribution = []
for _ in range(n_harmonics):
    envelop_1 = torch.sin(torch.linspace(0, np.random.uniform() * 40, time_steps)) + 1
    envelop_2 = torch.sin(torch.linspace(0, np.random.uniform() * 40, time_steps)) + 1
    envelop = torch.concat([envelop_1[None, :, None, None], envelop_2[None, :, None, None]], 2)
    harmonic_distribution.append(envelop)

harmonic_distribution = torch.concat(harmonic_distribution, dim=-1)
harmonic_distribution = torch.tile(harmonic_distribution, (batch_size, 1, 1, 1))

In [ ]:
with torch.no_grad():
    y = osc(base_pitch, amplitude, harmonic_distribution)

In [ ]:
y_np = y[0].numpy().T

In [ ]:
waveshow(y_np, sr=sample_rate)

In [ ]:
Audio(y_np, rate=sample_rate)

In [ ]:
noise = FilteredNoise(sample_rate, window_length, hop_length, n_bands, n_channels)

In [ ]:
filter_bands = []
for _ in range(n_bands):
    envelop_1 = torch.sin(torch.linspace(0, np.random.uniform() * 40, time_steps)) + 1
    envelop_2 = torch.sin(torch.linspace(0, np.random.uniform() * 40, time_steps)) + 1
    envelop = torch.concat([envelop_1[None, :, None, None], envelop_2[None, :, None, None]], 2)
    filter_bands.append(envelop)

filter_bands = torch.concat(filter_bands, dim=-1)

In [ ]:
y = noise(filter_bands)

In [ ]:
y_np = y[0, :].numpy().T

In [ ]:
waveshow(y_np, sr=sample_rate)

In [ ]:
Audio(y_np, rate=sample_rate)